# This notebook will be mainly used for the capstone project.

Project Title - (Applied Data Science Capstone Project) The Battle Of Neighbourhoods - Using City Segmentation and Clustering.

In [9]:
import pandas as pd
import numpy as np

In [10]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


In [11]:
import numpy as np 
import pandas as pd 
import json
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium
print('import successful')


import successful


In [44]:
from geopy.geocoders import Nominatim
import geopy
import geocoder
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
print('import successful')

import successful


scraping web data

In [13]:
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
data = data.text

In [14]:
soup = BeautifulSoup(data, 'html.parser')

In [15]:
postalCodelist = []
boroughList = []
neighbourhoodList = []

In [16]:
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodelist.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighbourhoodList.append(cells[2].text.rstrip('\n'))

In [17]:
toronto_df = pd.DataFrame({"PostalCode":postalCodelist, "Borough":boroughList,                      'neighbourhood':neighbourhoodList})

toronto_df.head()

,PostalCode,Borough,neighbourhood
0,M1A\n,Not assigned\n,Not assigned
1,M2A\n,Not assigned\n,Not assigned
2,M3A\n,North York\n,Parkwoods
3,M4A\n,North York\n,Victoria Village
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront"


Cleaning Data

In [18]:
toronto_df_dropna = toronto_df[toronto_df.Borough != 'Not assigned\n'].reset_index(drop = True)

toronto_df_dropna.head(10)
toronto = toronto_df_dropna

In [19]:
toronto = toronto.replace('\n','',regex=True)

In [20]:
toronto.head(10)

,PostalCode,Borough,neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


Grouping Neighbourhood in same Group

In [21]:
toronto_grouped = toronto.groupby(['PostalCode','Borough'], as_index = False).agg(lambda x: ",".join(x))

toronto = toronto_grouped
toronto.head(10)

,PostalCode,Borough,neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [22]:
for index, row in toronto.iterrows():
    if row['neighbourhood'] == "Not assigned":
        row['neighbourhood'] = row['Borough']

toronto.head(10)

,PostalCode,Borough,neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [23]:
column_names = ['PostalCode', 'Borough', 'neighbourhood']
test_df = pd.DataFrame(columns=column_names)

test_list = ['M5G', 'M2H', 'M4B', 'M1J', 'M4G', 'M4M', 'M1R', 'M9V', 'M9L', 'M5V', 'M1B', 'M5A']

for postcode in test_list:
    test_df = test_df.append(toronto[toronto['PostalCode']==postcode], ignore_index = True)

test_df

,PostalCode,Borough,neighbourhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,"Parkview Hill, Woodbine Gardens"
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,"Wexford, Maryvale"
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har..."


In [24]:
toronto.shape

(103, 3)

Loading Co-Ordinates from Coursera CSV file

In [25]:
coordinates = pd.read_csv("Geospatial_Coordinates.csv")
coordinates.head(5)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [26]:
coordinates.rename(columns = {"Postal Code": "PostalCode"}, inplace = True)
coordinates.head(10)

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


Merging Neighbourhood and Coordinates Table

In [27]:
toronto_loc = toronto.merge(coordinates, on = "PostalCode", how = "left")
toronto_loc.head(10)

,PostalCode,Borough,neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


Checking and Matching the coordinates added required by the question

In [38]:
column_names = ["PostalCode" , "Borough" , "neighbourhood" , "Latitude" , "Longitude"]
test_df = pd.DataFrame(columns=column_names)

test_list = ['M5G', 'M2H', 'M4B', 'M1J', 'M4G', 'M4M', 'M1R', 'M9V', 'M9L', 'M5V', 'M1B', 'M5A']

for postcode in test_list:
    test_df = test_df.append(toronto_loc[toronto_loc['PostalCode'] == postcode], ignore_index = True)

test_df.head(5)

,PostalCode,Borough,neighbourhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
